In [2]:
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_table
import dash_html_components as html
from dash.dependencies import Input, Output
import import_ipynb

In [20]:
!python3.7 Data_Preprocessing.ipynb

In [ ]:
fig = px.choropleth(locations=["CA", "TX", "NY", "DC"], locationmode="USA-states", color=[1,2,3,4], scope="usa")
fig.show()

In [ ]:
hospital = pd.read_csv("data/hospitalData.csv")
x1 = hospital[hospital["State"] == "United States"]
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = x1["Date"],
    y = x1["InpatBeds_Occ_AnyPat_LoCI"],
    mode = "lines",
    line_color = "blue",
    name = "95% CI Lower"))
fig.add_trace(go.Scatter(
    x = x1["Date"],
    y = x1["InpatBeds_Occ_AnyPat_UpCI"],
    fill = "tonexty",
    mode = "lines",
    line_color = "blue",
    name = "95% CI Upper"))
fig.add_trace(go.Scatter(
    x = x1["Date"],
    y = x1["InpatBeds_Occ_AnyPat_Est"],
    line_color = "black",
    name = "Estimate"
))
fig.add_trace(go.Scatter(
    x = x1["Date"],
    y = x1["InpatBeds_Occ_AnyPat_Est_Avail"],
    line_color = "red",
    name = "Estimated Availability"
))


fig.update_layout(hovermode = "x")

fig.show()

In [ ]:
usaData = pd.read_csv('data/usaData.csv')
usaData
fig = make_subplots(specs = [[{"secondary_y" : True}]])

fig.add_trace(
    go.Scatter(x = usaData["Date"], y = usaData["New Cases"], name = "New Cases"),
    secondary_y = False)
fig.add_trace(
    go.Scatter(x = usaData["Date"], y = usaData["New Deaths"], name = "New Deaths"),
    secondary_y = True)

In [ ]:
usaData = pd.read_csv('data/usaData.csv')
usaData

x = usaData[usaData["%Retail/Rec Change"].isnull() == False]
x

fig2 = make_subplots(specs = [[{"secondary_y" : True}]])

fig2.add_trace(go.Scatter(
    x = x["Date"], y = x['New Cases'], name = "New Cases", line = dict(color = "black", dash = "dot", width = 4),
), secondary_y = False)

fig2.add_trace(go.Scatter(
    x = x["Date"], y = x['%Retail/Rec Change'], name = "Change in Retail/Recreation", line = dict(color = "yellow"),
), secondary_y = True)

fig2.add_trace(go.Scatter(
    x = x["Date"], y = x['%Grocery/Pharm Change'], name = "Change in Grocery/Pharmacy", line = dict(color = "red"),
), secondary_y = True)

fig2.add_trace(go.Scatter(
    x = x["Date"], y = x['%Parks Change'], name = "Change in Parks", line = dict(color = "green"),
), secondary_y = True)

fig2.add_trace(go.Scatter(
    x = x["Date"], y = x['%Transit Change'], name = "Change in Transit Stations", line = dict(color = "blue"),
), secondary_y = True)

fig2.add_trace(go.Scatter(
    x = x["Date"], y = x['%Workplace Change'], name = "Change in Workplaces", line = dict(color = "purple"),
), secondary_y = True)

fig2.add_trace(go.Scatter(
    x = x["Date"], y = x['%Residential Change'], name = "Change in Residential", line = dict(color = "orange"),
), secondary_y = True)

fig.update_layout(
    title = "Changes in Mobility",
    xaxis = dict(title_text = "Date", title_standoff = 50),
    yaxis_title = "% Change from Baseline"
)

In [ ]:
v = pd.read_csv("data/demoDeaths.csv")
x = v[v["State"] == "United States"]
np.max(x["COVID-19 Deaths"])

In [ ]:
v = pd.read_csv("data/demoDeaths.csv")
x = v[(v["State"] == "United States") & (v["Sex"] != "All")]
x

fig = go.Figure(data = [
        go.Bar(name = "Male", x = x["Age group"], y = x['COVID-19 Deaths'][x["Sex"] == "Male"],
              text = x['COVID-19 Deaths'][x["Sex"] == "Male"], textposition = 'auto'),
        go.Bar(name = "Female", x = x["Age group"], y = x['COVID-19 Deaths'][x["Sex"] == "Female"],
              text = x['COVID-19 Deaths'][x["Sex"] == "Female"], textposition = 'auto')])
fig.update_layout(barmode = "group")

fig.show()

In [ ]:
v = pd.read_csv("data/raceDeaths.csv")
x = v[v["State"] == "United States"]
x

# colors = ['crimson','green','yellow','blue','orange','purple']

# fig = go.Figure(data = [go.Bar(
#     x = x["Race"], y = x["Count of COVID-19 deaths"],
#     text = x["Count of COVID-19 deaths"],
#     textposition = "auto",
#     marker_color = colors)],)

# fig.show()

Use multiprocessing to: 
1) Calculate the number of new cases each day.

2) Calculate the number of new deaths each day.

In [ ]:
### Import Pool from multiprocessing
from multiprocessing import Pool

In [ ]:
### Create a parallelizing function
def parallel(data, func, n_cores = 4):
    ### Split data by state into 4 sections
    splits = np.array_split(cases_deaths["State"].unique(), 4)
    split1 = cases_deaths[cases_deaths["State"].isin(list(splits[0]))]
    split2 = cases_deaths[cases_deaths["State"].isin(list(splits[1]))]
    split3 = cases_deaths[cases_deaths["State"].isin(list(splits[2]))]
    split4 = cases_deaths[cases_deaths["State"].isin(list(splits[3]))]
    
    ### Combine into list
    data_split = [split1, split2, split3, split4]
    
    pool = Pool(n_cores)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

In [ ]:
### Define function to create new cases data
def newCases(data):
    changeInCases = []
    ### For each state.
    for state in data["State"].unique():
        ### For each county in the state
        for county in data["County Name"][data["State"] == state].unique():
            ### Calculate diff in case for each day, keep first day
            changeInCases.extend(abs(np.diff(data["Total Cases"][(data["County Name"] == county) &
                                                                         (data["State"] == state)],
                                             prepend = data["Total Cases"][(data["County Name"] == county) &
                                                                         (data["State"] == state)].iloc[0])))
    ### Add to data
    data["New Cases"] = changeInCases

    return data

In [ ]:
cases_deaths = parallel(cases_deaths, newCases)
cases_deaths

In [ ]:
### Define function to create new deaths data
def newDeaths(data):
    changeInDeaths = []
    ### For each state.
    for state in data["State"].unique():
        ### For each county in the state
        for county in data["County Name"][data["State"] == state].unique():
            ### Calculate diff in case for each day, keep first day
            changeInDeaths.extend(abs(np.diff(data["Total Deaths"][(data["County Name"] == county) &
                                                                           (data["State"] == state)],
                                             prepend = data["Total Deaths"][(data["County Name"] == county) &
                                                                           (data["State"] == state)].iloc[0])))
            
    ### Add to data
    data["New Deaths"] = changeInDeaths
        
    return data

In [ ]:
cases_deaths = parallel(cases_deaths, newDeaths)
cases_deaths